In [1]:
from openai import OpenAI

In [2]:
client = OpenAI()

In [3]:
import minsearch

In [4]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]
)

In [ ]:
SELECT * WHERE course='data-engineering-zoomcamp';

In [7]:
q = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
boost = {
    'question': 3.0,
    'section': 0.5    
}

results = index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [10]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [11]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user', 'content':q}]
)

In [12]:
response

ChatCompletion(id='chatcmpl-Cuh7PFy8fJMuvg6UGHt4IxIXeCerq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Whether you can still enroll in a course that has already started depends on a few factors:\n\n1. **Institution's Policy:** Some institutions allow late enrollment within a certain period after the course has begun, while others have strict deadlines.\n\n2. **Type of Course:** If it's an online, self-paced course, you may be able to join at any time. However, for structured courses with a strict schedule, enrolling late might not be possible.\n\n3. **Instructor's Decision:** Sometimes, the course instructor may have the discretion to allow late enrollments based on specific circumstances.\n\n4. **Availability of Resources:** If the course is full or there are limited resources (like lab access), late enrollment might not be feasible.\n\nTo find out if you can still enroll, contact the course organizer, instructor, or the admiss

In [13]:
response.choices[0].message.content

"Whether you can still enroll in a course that has already started depends on a few factors:\n\n1. **Institution's Policy:** Some institutions allow late enrollment within a certain period after the course has begun, while others have strict deadlines.\n\n2. **Type of Course:** If it's an online, self-paced course, you may be able to join at any time. However, for structured courses with a strict schedule, enrolling late might not be possible.\n\n3. **Instructor's Decision:** Sometimes, the course instructor may have the discretion to allow late enrollments based on specific circumstances.\n\n4. **Availability of Resources:** If the course is full or there are limited resources (like lab access), late enrollment might not be feasible.\n\nTo find out if you can still enroll, contact the course organizer, instructor, or the admissions office as soon as possible. They will provide guidance based on the specific course and institution policies."

In [14]:
prompt_template = """
You're a course teaching assisstant. Answer the QUESTION based on the CONTEXT from the FAQ Database.
Use only the fact from the CONTEXT when answering QUESTION.
If the CONTEXT don't contain the answer, output NONE.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [15]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [16]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [17]:
prompt = prompt_template.format(question=q, context=context).strip()

In [18]:
print(prompt)

You're a course teaching assisstant. Answer the QUESTION based on the CONTEXT from the FAQ Database.
Use only the fact from the CONTEXT when answering QUESTION.
If the CONTEXT don't contain the answer, output NONE.

QUESTION: the course has already started, can I still enroll?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final c

In [19]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user', 'content':prompt}]
)
response.choices[0].message.content

'Yes, you can still join the course after it has started.'

In [20]:
def search(query):
    boost = {
        'question': 3.0,
        'section': 0.5    
    }

    results = index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=10
    )

    return results

In [21]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assisstant. Answer the QUESTION based on the CONTEXT from the FAQ Database.
    Use only the fact from the CONTEXT when answering QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()    

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [22]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role':'user', 'content':prompt}]
    )
    return response.choices[0].message.content

In [23]:
query = 'how do I run Kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [24]:
rag(query)

'To run Kafka, you need to ensure that your Kafka broker docker container is up and running. You can confirm this by using the command `docker ps`. If it is not working, navigate to the docker compose YAML file folder and run `docker compose up -d` to start all the instances.'

In [25]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course after it has started. You are eligible to submit the homework even if you haven't registered. However, be mindful of the deadlines for submitting the final projects, so it's best not to leave everything until the last minute."

In [26]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [28]:
from elasticsearch import Elasticsearch

In [29]:
es_client = Elasticsearch('http://localhost:9200')

In [30]:
es_client.info()

ObjectApiResponse({'name': '2e2108e2469a', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'XtiNkdNCT3a7vav-75BxhQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [31]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

/tmp/ipykernel_33021/295183685.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [32]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████| 948/948 [00:04<00:00, 236.16it/s]


In [35]:
query = 'I just discovered the course. Can I still join it?'

In [44]:
def elastic_search(query):
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    } 

    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [38]:
response = es_client.search(index=index_name, body=search_query)

/tmp/ipykernel_33021/449040615.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


In [40]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': '0pfhjpsBo6YYjHdLlEBU',
  '_score': 72.849266,
  '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
   'section': 'General course-related questions',
   'question': 'Course - Can I still join the course after the start date?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': '15fhjpsBo6YYjHdLlEB6',
  '_score': 54.057133,
  '_source': {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
   'section': 'General course-related questions',
   'question': 'Course - Can I follow the course after 

In [41]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [42]:
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [45]:
elastic_search(query)

/tmp/ipykernel_33021/4154412784.py:23: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [46]:
def rag_elastic(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [47]:
rag_elastic(query)

/tmp/ipykernel_33021/4154412784.py:23: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


"Yes, you can still join the course. Even if you don't register, you're still eligible to submit the homeworks. However, be mindful of the deadlines for turning in the final projects."